### Topic - Using tools
##### Enable models to fetch data and take actions.

In [33]:
import json
import os

import requests
from openai import OpenAI
from pydantic import BaseModel, Field

In [34]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## 🛠️ Function Calling

**Function Calling**: Lets the AI call your backend functions with structured arguments based on the user's request.

- Allows dynamic interaction between AI and external tools or APIs.
- Useful for tasks like retrieving data, triggering actions, or generating responses based on real-time inputs.


---
**Introduction:** Using a Weather API, we can model it as a tool and make it available to the LLM. Based on the context, the LLM looks at the available tools and, depending on the user's input or question, decides whether it wants to use the tool or not.

##### Define the tool (function) that we want to call


In [35]:
def get_weather(latitude, longitude):
    """This is a publically available API that returns the weather for a given location."""
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
    )
    data = response.json()
    return data["current"]

#### Step 1: Call model with get_weather tool defined


In [36]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current temperature for provided coordinates in celsius.",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {"type": "number"},
                    "longitude": {"type": "number"},
                },
                "required": ["latitude", "longitude"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    }
]


In [ ]:
system_prompt = "You are a helpful weather assistant."

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "What's the weather like in Paris today?"},
]


In [ ]:
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

#### Step 2: Model decides to call function(s)


In [40]:
completion.model_dump()

{'id': 'chatcmpl-BT270WskdHzlbZppBxs08dIJR1suJ',
 'choices': [{'finish_reason': 'tool_calls',
   'index': 0,
   'logprobs': None,
   'message': {'content': None,
    'refusal': None,
    'role': 'assistant',
    'annotations': [],
    'audio': None,
    'function_call': None,
    'tool_calls': [{'id': 'call_gkAe4N00kfEKowOECXHM7rf6',
      'function': {'arguments': '{"latitude":48.8566,"longitude":2.3522}',
       'name': 'get_weather'},
      'type': 'function'}]}}],
 'created': 1746259330,
 'model': 'gpt-4o-2024-08-06',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_d8864f8b6b',
 'usage': {'completion_tokens': 25,
  'prompt_tokens': 66,
  'total_tokens': 91,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}}

In [41]:
tool_calls = completion.choices[0].message.tool_calls
formatted_tool_call = json.dumps(tool_calls[0].model_dump(), indent=4)
print(formatted_tool_call)


{
    "id": "call_gkAe4N00kfEKowOECXHM7rf6",
    "function": {
        "arguments": "{\"latitude\":48.8566,\"longitude\":2.3522}",
        "name": "get_weather"
    },
    "type": "function"
}


#### Step 3: Execute get_weather function

**Important Note:** The LLM is not going to call the tool for you. It will just return the function call with the parameters that you need to pass to the function. 

In [42]:
def call_function(name, args):
    if name == "get_weather":
        return get_weather(**args)

In [43]:
for tool_call in tool_calls:
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    messages.append(completion.choices[0].message)

    result = call_function(name, args)
    messages.append(
        {"role": "tool", "tool_call_id": tool_call.id, "content": json.dumps(result)}
    )

In [44]:
name

'get_weather'

In [45]:
args

{'latitude': 48.8566, 'longitude': 2.3522}

In [46]:
messages

[{'role': 'system', 'content': 'You are a helpful weather assistant.'},
 {'role': 'user', 'content': "What's the weather like in Paris today?"},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_gkAe4N00kfEKowOECXHM7rf6', function=Function(arguments='{"latitude":48.8566,"longitude":2.3522}', name='get_weather'), type='function')]),
 {'role': 'tool',
  'tool_call_id': 'call_gkAe4N00kfEKowOECXHM7rf6',
  'content': '{"time": "2025-05-03T08:00", "interval": 900, "temperature_2m": 20.0, "wind_speed_10m": 6.5}'}]

In [54]:
result

{'time': '2025-05-03T08:00',
 'interval': 900,
 'temperature_2m': 20.0,
 'wind_speed_10m': 6.5}

#### Step 4: Supply result and call model again

In [48]:
class WeatherResponse(BaseModel):
    temperature: float = Field(
        description="The current temperature in celsius for the given location."
    )
    response: str = Field(
        description="A natural language response to the user's question."
    )

In [49]:
completion_2 = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    response_format=WeatherResponse,
)



#### Step 5: Check model response

In [50]:
final_response = completion_2.choices[0].message.parsed

In [51]:
final_response.model_dump()

{'temperature': 20.0,
 'response': 'The current temperature in Paris is 20°C with a light breeze of wind speed 6.5 m/s.'}

In [52]:
final_response.temperature

20.0

In [53]:
final_response.response

'The current temperature in Paris is 20°C with a light breeze of wind speed 6.5 m/s.'

> 📖 More details of Function Calling on the [OpenAI Docs](https://platform.openai.com/docs/guides/function-calling)



## LATEST

In [1]:
from openai import OpenAI

client = OpenAI()

In [2]:
tools = [
    {
        "type": "function",
        "name": "send_email",
        "description": "Send an email to a given recipient with a subject and message.",
        "parameters": {
            "type": "object",
            "properties": {
                "to": {"type": "string", "description": "The recipient email address."},
                "subject": {"type": "string", "description": "Email subject line."},
                "body": {"type": "string", "description": "Body of the email message."},
            },
            "required": ["to", "subject", "body"],
            "additionalProperties": False,
        },
    }
]

In [3]:
response = client.responses.create(
    model="gpt-4o",
    input="Can you send an email to ilan@example.com and katia@example.com saying hi?",
    tools=tools,
)

In [4]:
print(response.output)

[ResponseFunctionToolCall(arguments='{"to":"ilan@example.com","subject":"Hello","body":"Hi"}', call_id='call_a2xmgEOrhlcvBCoEdUZD1ZdG', name='send_email', type='function_call', id='fc_68185cc4a65c8191b77e77c5767021270abf586043ed9ae1', status='completed'), ResponseFunctionToolCall(arguments='{"to":"katia@example.com","subject":"Hello","body":"Hi"}', call_id='call_GWQWCVQvS7KaaDh2yOWB3WCP', name='send_email', type='function_call', id='fc_68185cc4c9208191a68d077178dc728e0abf586043ed9ae1', status='completed')]


In [5]:
print(response.output[0].model_dump_json(indent=2))

{
  "arguments": "{\"to\":\"ilan@example.com\",\"subject\":\"Hello\",\"body\":\"Hi\"}",
  "call_id": "call_a2xmgEOrhlcvBCoEdUZD1ZdG",
  "name": "send_email",
  "type": "function_call",
  "id": "fc_68185cc4a65c8191b77e77c5767021270abf586043ed9ae1",
  "status": "completed"
}


In [6]:
print(response.output[1].model_dump_json(indent=2))

{
  "arguments": "{\"to\":\"katia@example.com\",\"subject\":\"Hello\",\"body\":\"Hi\"}",
  "call_id": "call_GWQWCVQvS7KaaDh2yOWB3WCP",
  "name": "send_email",
  "type": "function_call",
  "id": "fc_68185cc4c9208191a68d077178dc728e0abf586043ed9ae1",
  "status": "completed"
}
